<div class="admonition info">
    <p class="admonition-title">Info</p>
    <p>Click <a href="https://mybinder.org/v2/gh/oqtopus-team/tranqu/HEAD?labpath=docs/usage/how_it_works.ipynb" target="_blank">here</a> to run this notebook interactively on Binder.</p>
</div>

# How It Works

This page provides an overview of how Tranqu performs transpilation and explains the internal processing steps.

## What is Tranqu?

Tranqu is is a one-stop framework for transpilers across multiple quantum programming libraries and formats for quantum circuits.

Quantum circuit transpilation is NP-complete, and heuristic algorithms are typically used.
While many transpiler libraries exist, the optimal transpiler and its options often depend on the quantum circuit and device information.
Therefore, the ability to leverage various vendors' transpilers is highly desirable.
By using Tranqu, you can run various vendors' transpilers without being bothered by the conversion of quantum circuits or device information.

## How Tranqu's transpile function works

The `transpile` function of Tranqu performs transpilation using the specified library, taking a quantum program and device information as inputs.

The input quantum program is set to the `program` argument, and the input device information is set to the `device` argument.
To specify the format, set the `program_lib` argument when setting the program.
Similarly, set the `device_lib` argument when setting the device.

The transpiler library is set to the `transpiler_lib` argument.
Tranqu does not perform transpilation directly but utilizes other transpiler libraries to execute the transpilation process.
The transpiler options are set as a dict in the `transpiler_options` argument.

The following code is a sample that sets all of these arguments.

In [1]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from tranqu import Tranqu

tranqu = Tranqu()

# prepare program
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()

# prepare device
backend = FakeManilaV2()

# prepare transpiler options
options = {
    "basis_gates": ["id", "sx", "x", "rz", "cx"]
}

result = tranqu.transpile(
    program=circuit,
    program_lib="qiskit",
    transpiler_lib="qiskit",
    transpiler_options=options,
    device=backend,
    device_lib="qiskit",
)
print(result.transpiled_program)

2
global phase: π/4
               ┌─────────┐┌────┐┌─────────┐      ░ ┌─┐   
      q_0 -> 0 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■───░─┤M├───
               └─────────┘└────┘└─────────┘┌─┴─┐ ░ └╥┘┌─┐
      q_1 -> 1 ────────────────────────────┤ X ├─░──╫─┤M├
                                           └───┘ ░  ║ └╥┘
ancilla_0 -> 2 ─────────────────────────────────────╫──╫─
                                                    ║  ║ 
ancilla_1 -> 3 ─────────────────────────────────────╫──╫─
                                                    ║  ║ 
ancilla_2 -> 4 ─────────────────────────────────────╫──╫─
                                                    ║  ║ 
       meas: 2/═════════════════════════════════════╩══╩═
                                                    0  1 


The minimum required arguments are `program`, `program_lib`, and `transpiler_lib`.
In this case, `transpiler_options` is set to the default options of the transpiler library, and transpilation proceed without utilizing any device information.

In [2]:
result = tranqu.transpile(
    program=circuit,
    program_lib="qiskit",
    transpiler_lib="qiskit",
)
print(result.transpiled_program)

        ┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1 


You can set the built-in options provided by Tranqu for `transpiler_lib`, `program_lib`, and `device_lib`.
For details on the built-in components, refer to [Built-in Components](#built-in-components).

Furthermore, you can set custom transpilers, custom programs, and custom devices that you have created.
For details on how to create and use custom options, refer to [Using Custom Transpilers and Converters](../custom).

## Inside the transpile function

Inside the transpile function, the following steps are executed:

### Step 1: Fetch the transpiler library
The transpilers available in Tranqu are registered in the `TranspilerManager` class.  
Using the `transpiler_lib` argument, the appropriate transpiler library is fetched from the `TranspilerManager`.
If the `default_transpiler_lib` is set, the `transpiler_lib` argument can be omitted.

### Step 2: Fetch the `ProgramConverter`
Tranqu uses the `ProgramConverter` class to convert the format of the input program.  
To transform the input program into a format that can be processed by the specified transpiler, the appropriate `ProgramConverter` is fetched from the `ProgramConverterManager` class using the `program_lib` and `transpiler_lib` arguments.
If the `program_type` is set, the `program_lib` argument can be omitted.

### Step 3: Fetch the `DeviceConverter`
Tranqu uses the `DeviceConverter` class to convert the format of the input device information.  
To transform the input device information into a format that can be processed by the specified transpiler, the appropriate `DeviceConverter` is fetched from the `DeviceConverterManager` class using the `device_lib` and `transpiler_lib` arguments.  
If the `device_type` is set, the `device_lib` argument can be omitted.

If `device_lib` is not set, this step is skipped.

### Step 4: Execute the transpilation
Using the program and device information converted into a format compatible with the transpiler, the transpilation process is executed.

### Step 5: Post-processing
Retrieve the mapping between virtual qubit (or bit) indices and physical qubit (or bit) indices before and after transpilation.
Additionally, obtain statistical information such as the number of gates before and after transpilation.  

Additionally, since the transpiled program is in the format specified by `transpiler_lib`, it is converted into the format specified by `program_lib`.

## Built-in Components

When `Tranqu` class is instantiated, it automatically registers `Transpiler`, `ProgramConverter`, and `DeviceConverter` to their respective Managers.  
These components are referred to as "built-in components."

Table 1: Built-in `Transpiler`

| Transpiler Library | Description                               |
|--------------------|-------------------------------------------|
| `"qiskit"`         | Transpile quantum circuits using Qiskit.  |
| `"ouqu-tp"`        | Transpile quantum circuits using ouqu-tp. |

<div class="admonition info">
    <p class="admonition-title">Info</p>
    <p>ouqu-tp. To use ouqu-tp, <a href="https://github.com/softwareQinc/staq/blob/main/INSTALL.md" target="_blank">staq</a> is required. If staq is not installed, it will be automatically installed the first time you use ouqu-tp. The installation of staq takes several minutes.</p>
</div>

Table 2: Built-in `ProgramConverter`

| From          | To            | Description                        |
|---------------|---------------|------------------------------------|
| `"openqasm3"` | `"qiskit"`    | Converts from OpenQASM3 to Qiskit. |
| `"openqasm3"` | `"tket"`      | Converts from OpenQASM3 to tket.   |
| `"qiskit"`    | `"openqasm3"` | Converts from Qiskit to OpenQASM3. |
| `"qiskit"`    | `"tket"`      | Converts from Qiskit to tket.      |
| `"tket"`      | `"openqasm3"` | Converts from tket to OpenQASM3.   |
| `"tket"`      | `"qiskit"`    | Converts from tket to Qiskit.      |


Table 3: Built-in `DeviceConverter`

| From        | To         | Description                      |
|-------------|------------|----------------------------------|
| `"oqtopus"` | `"qiskit"` | Converts from OQTOPUS to Qiskit. |

## Built-in types

When `Tranqu` class is instantiated, it automatically registers `program_type`, and `device_type` to their respective Managers.  
These components are referred to as "built-in types."

Table 4: Built-in `program_type`

| Program Library | Type                    |
|-----------------|-------------------------|
| `"qiskit"`      | `qiskit.QuantumCircuit` |
| `"tket"`        | `pytket.Circuit`        |


Table 5: Built-in `device_type`

| Device Library | Type                         |
|----------------|------------------------------|
| `"qiskit"`     | `qiskit.providers.BackendV2` |